#### Emoji prediction

In [57]:
## Libraries
import numpy as np
import pandas as pds
from keras.layers import LSTM,Dense
from keras.models import Sequential
from keras.utils import to_categorical
import emoji

In [58]:
## Extracting the data
data_X = pds.read_csv('train_emoji.csv',header=None)
data_X.head(n=10)

## Making input and output
X = data_X[0].values
Y = data_X[1].values
print("The shape of X is {}".format(X.shape))
print("The shape of Y is {}".format(Y.shape))

The shape of X is (132,)
The shape of Y is (132,)


In [59]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [60]:
## Visulizing the emoji
for i in range(10):
    print(X[i],emoji.emojize(emoji_dictionary[str(Y[i])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [61]:
## Creating thr word-vec
## Reading the vector
embedding = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        embedding[word] = coeffs

In [62]:
## Preprocess data
X_ = np.zeros((X.shape[0],10,50))
for i in range(X.shape[0]):
    X_split = X[i].lower().split()
    for j in range(len(X_split)):
        X_[i][j] = embedding[X_split[j]]
        
Y_ = to_categorical(Y)
print(Y_.shape)
print(X_[0].shape)

(132, 5)
(10, 50)


In [63]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [64]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(X_,Y_,epochs=10)

Epoch 1/10
132/132 [==============================] - 1s 11ms/step - loss: 1.6062 - acc: 0.2652
Epoch 2/10
132/132 [==============================] - 0s 689us/step - loss: 1.5293 - acc: 0.3409
Epoch 3/10
132/132 [==============================] - 0s 1ms/step - loss: 1.5058 - acc: 0.3182
Epoch 4/10
132/132 [==============================] - 0s 1ms/step - loss: 1.4690 - acc: 0.3182
Epoch 5/10
132/132 [==============================] - 0s 981us/step - loss: 1.4449 - acc: 0.3106
Epoch 6/10
132/132 [==============================] - 0s 940us/step - loss: 1.3671 - acc: 0.4091
Epoch 7/10
132/132 [==============================] - 0s 784us/step - loss: 1.2868 - acc: 0.5606
Epoch 8/10
132/132 [==============================] - 0s 739us/step - loss: 1.1835 - acc: 0.5455
Epoch 9/10
132/132 [==============================] - 0s 727us/step - loss: 1.1233 - acc: 0.6212
Epoch 10/10
132/132 [==============================] - 0s 640us/step - loss: 0.9924 - acc: 0.7121


In [65]:
model.evaluate(X_,Y_)

132/132 [==============================] - 0s 2ms/step


[0.8764923503904631, 0.7878788113594055]

In [66]:
y_Pred = model.predict_classes(X_)

## Visulizing the emoji
for i in range(10):
    print(X[i],"\n",emoji.emojize(emoji_dictionary[str(y_Pred[i])]),"\n",emoji.emojize(emoji_dictionary[str(Y[i])]))

never talk to me again 
 😓 
 😓
I am proud of your achievements 
 😁 
 😁
It is the worst day in my life 
 😓 
 😓
Miss you so much 
 ❤️ 
 ❤️
food is life 
 🍴 
 🍴
I love you mum 
 ❤️ 
 ❤️
Stop saying bullshit 
 😓 
 😓
congratulations on your acceptance 
 😁 
 😁
The assignment is too long  
 😓 
 😓
I want to go play 
 ⚾ 
 ⚾


In [67]:
model.save_weights('model.h5')

In [68]:
model.load_weights('model.h5')
model.evaluate(X_,Y_)

132/132 [==============================] - 0s 176us/step


[0.8764923503904631, 0.7878788113594055]